In [7]:
# Import the necessary dependencies.
from flask import Flask, jsonify
from datetime import datetime, timedelta
from sqlalchemy import create_engine, func
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base
import os

In [2]:
#################################################
# Database Setup
#################################################

# Create an engine to connect to the SQLite database file.
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

# Reflect the tables and create references to each table.
Base = automap_base()
Base.prepare(engine, reflect=True)
Measurement = Base.classes.measurement
Station = Base.classes.station

# Create a session to establish a link from Python to the database.
session = Session(engine)

In [9]:
#################################################
# Flask Setup
#################################################

app = Flask(__name__)

#################################################
# Flask Routes
#################################################

@app.route("/")
def welcome():
    return (
        f"Welcome Page<br/>"
        f"Routes:<br/>"
        f"/api/v1.0/precipitation<br/>"
        f"/api/v1.0/stations<br/>"
        f"/api/v1.0/tobs<br/>"
        f"/api/v1.0/<start>/<end><br/>"
    )

@app.route("/api/v1.0/precipitation")
def precipitation():
    last_date = session.query(Measurement.date).order_by(Measurement.date.desc()).first()
    date_str = last_date[0]
    date_object = datetime.strptime(date_str, '%Y-%m-%d')
    one_year_ago = date_object - timedelta(days=365)
    formatted_result = one_year_ago.strftime('%Y-%m-%d')
    year_prcp = session.query(Measurement.date, Measurement.prcp).\
        filter(Measurement.date >= formatted_result, Measurement.prcp != None).\
        order_by(Measurement.date).all()
    return jsonify(dict(year_prcp))

@app.route("/api/v1.0/stations")
def stations():
    results = session.query(Station.station).all()
    stations_list = list(np.ravel(results))
    return jsonify(stations_list)

@app.route("/api/v1.0/tobs")
def tobs():
    tobss = session.query(Measurement.tobs).\
        filter(Measurement.station == 'USC00519281').\
        filter(Measurement.date >= '2017-08-23').all()
    tobs_list = list(np.ravel(tobss))
    return jsonify(tobs_list)

@app.route("/api/v1.0/<start>")
@app.route("/api/v1.0/<start>/<end>")
def temperatures(start, end=None):
    if end is None:
        end = session.query(Measurement.date).order_by(Measurement.date.desc()).first()[0]
    results = session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start, Measurement.date <= end).all()
    return jsonify({
        "start_date": start,
        "end_date": end,
        "min_temperature": results[0][0],
        "avg_temperature": results[0][1],
        "max_temperature": results[0][2]
    })

# if __name__ == "__main__":
#     app.run(debug=True)

    
if __name__ == "__main__":
    app.run(host=os.getenv('IP', '0.0.0.0'), 
            port=int(os.getenv('PORT', 4444)),
            debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


Address already in use
Port 4444 is in use by another program. Either identify and stop that program, or start the server with a different port.


AttributeError: 'tuple' object has no attribute 'tb_frame'